# Итераторы
В этом лонгриде рассмотрим, что такое объекты-итераторы, и как с ними работать

## цикл for в Python

Цикл for в других языках программирования обычно перебирает числовые индексы от ... до ... с шагом ...

Как, например, в языке C++

<img src="imgs/for_cycle.png" width="600"/>

Цикл for, в Python, устроен несколько иначе, чем в большинстве других языков
он больше похож на конструкции for...each, или же for...of


### Перебор коллекций

Однако чаще всего задача цикла - сделать перебор элементов какой-то коллекции (массива, списка, словаря и т.д.). При этом:
- получить доступ к содержимому коллекции без раскрытия их внутреннего представления;
- иметь возможность перебрать эту коллекцию несколько раз;
- создать единый "интерфейс" для обхода различных колеекций.

А в качестве такого интерфейса может выступать, например, цикл for.

### Итератор

Итератор - объект, который позволяет перебрать элементы коллекции по указанным выше правилам. Коллекция, элементы которой перебираются итератором, называется **агрегатом**.

Согласно определению, данному в книге "Банды четырёх", итератор - это объект, для которого определены следующие методы:

- `first`, возвращающий итератор к началу агрегата
- `next`, переводящий итератор на следующий элемент агрегата.
- `isDone`, определяющий, не закончились ли элементы в агрегате
- `currentItem`, возвращающий текущий элемент агрегата, "перехваченный" итератором

### Итераторы в Python

В Python достаточно, чтобы в объекте был реализован метод `__next__`, чтобы он стал итератором.

In [1]:
# создадим итератор из списка
x = iter([1,2,3,4,5])

In [2]:
next(x)

1

In [3]:
next(x)

2

In [4]:
next(x)

3

Причём `next` применим только к итераторам

In [5]:
next(5)

TypeError: 'int' object is not an iterator

Созданный итератор может двигаться только вперёд: каждый вызов `next` возвращает следующий элемент агрегата.

Например, если попросить цикл перебрать элементы итератора, он вернёт только "оставшиеся"

In [6]:
for i in x:
    print(i)

4
5


In [7]:
# итератор "закончился"
# при попытке получить следующий элемент будет возвращено исключение
next(x)

StopIteration: 

### При чём тут цикл for?
Цикл `for` на каждом шаге вызывает метод `next`, проверяя, не вернул ли итератор исключение `StopIteration`. В случае появления исключения `for` завершает свою работу. Например, это может случиться, если элементы агрегата закончились

Но ведь в цикл `for` можно подставить, например, список

In [8]:
l = [1,2,3,4,5]
for i in l:
    print(i)

1
2
3
4
5


### Почему в цикл for можно подставить список или кортеж?
Разве список - это итератор?

In [9]:
# не похоже
next(l)

TypeError: 'list' object is not an iterator

## Итерируемый объект
Некоторые объекты в Python позволяют автоматически неявно создавать на своей основе итераторы. Обычно объекты таких классов мы и называем коллекциями.

Чтобы объекты класса стали итерируемые, в классе необходимо определить метод `__iter__!`.

Создадим свой итерируемый объект, и итератор, который будет создаваться на его основе

In [10]:
import collections

# класс итерируемых объектов
class ListCollection(collections.abc.Iterable):
    def __init__(self, collection):
        self._collection = collection

    def __iter__(self):
        return ListIterator(self._collection, -1)

In [11]:
# класс итераторов
class ListIterator(collections.abc.Iterator):
    def __init__(self, collection, cursor):
        self._collection = collection
        self._cursor = cursor

    def __next__(self):
        if self._cursor + 1 >= len(self._collection):
            raise StopIteration()
        self._cursor += 1
        return self._collection[self._cursor]

In [12]:
# создадим итерируемый объект
custom_list = ListCollection([1,2,3,4,5])

In [13]:
# по нему самому нельзя итерироваться
next(custom_list)

TypeError: 'ListCollection' object is not an iterator

In [14]:
# однако цикл for будет неявно создавать итератор из этого объекта
for i in custom_list:
    print(i)

1
2
3
4
5


In [15]:
# а мы можем создать его явно с помощью метода next
custom_iterator = iter(custom_list)

In [16]:
# кстати, iter можно применить и с помощью обычного синтаксиса для применения методов
# результат будет тот же самый
custom_list = custom_list.__iter__()

In [17]:
next(custom_list)

1

In [18]:
next(custom_list)

2

In [19]:
# цикл "допереберёт" оставшиеся в итераторе элементы
for i in custom_list:
    print(i)

3
4
5


### Итератор - это тоже итерируемый объект
По умолчанию, из одного итератора можно создать другой итератор

In [20]:
custom_iterator_from_iterator = iter(custom_iterator)

In [21]:
next(custom_iterator_from_iterator)

1

In [22]:
next(custom_iterator_from_iterator)

2

## Полезные итераторы
Многие встроенные в Python функции возвращают объекты-итераторы.

Например, `enumerate`, `zip`, `reversed`, `map` и другие

## enumerate
возвращает итератор, перебирающий пары ("индекс элемента коллекции", "сам элемент коллекции")

In [23]:
enum = enumerate(['zero element','first element','second element'])
enum

In [24]:
next(enum)

(0, 'zero element')

In [25]:
# заметьте, тут напечатаются уже только "оставшееся" в итераторе элементы
for index, element in enum:
    print(index, element)

1 first element
2 second element


In [26]:
# если мы попробуем ещё раз пройти циклом по элементам итератора, ничего не произойдёт
# итератор необходимо будет создавать заново
for index, element in enum:
    print(index, element)

## zip
возвращает итератор, перебирающий упорядоченные пары из элементов двух коллекций

Например, для множеств X и Y это будут пары

(x0, y0)

(x1, y1)

и т.д.

In [27]:
x = ['zero element X','first element X','second element X']
y = ['zero element Y','first element Y','second element Y']
zip_iter = zip(x, y)
zip_iter

In [28]:
for index, element in zip_iter:
    print(index, ";", element)

zero element X ; zero element Y
first element X ; first element Y
second element X ; second element Y


### reversed
возвращает итератор, перебирающий элементы коллекции в обратном порядке

In [29]:
r = reversed([1,2,3])
r

In [30]:
for i in r:
    print(i)

3
2
1


### map
возвращает итератор, перебирающий элементы коллекции, к которым применена функция

In [31]:
m = map(lambda x: x**2, [1,2,3])
m

In [32]:
next(m)

1

In [33]:
next(m)

4

In [34]:
next(m)

9

## Полезные итераторы itertools
В модуле `itertools` содержится большое число полезных итераторов. Особенно позволяющих работать с гипотетически неограниченными последовательностями

In [35]:
import itertools

### "Бесконечный" итератор
Возвращает **n, n+1, n+2, ...**

In [36]:
counter = itertools.count(5)

In [37]:
next(counter)

5

In [38]:
next(counter)

6

In [39]:
next(counter)

7

### "Бесконечный" зацикливающий коллекцию итератор
Циклически перебирает элементы переданной ему коллекции

In [40]:
l = [1,2,3]
cycle = itertools.cycle(l)

In [41]:
for i in range(10):
    print(next(cycle))

1
2
3
1
2
3
1
2
3
1


## range - не итератор
Функция `range`, позволяющая удобно перебирать числа "от - до - с шагом", не является итератором или генератором (см. следующую лекцию).

Она вычисляет свои значения "лениво", как и генераторы. Однако, реализация range отличается от генераторов.

К ним не применим метод `next`, но применим цикл `for`. `range` создаёт "ленивые итерируемые объекты".

Подробнее об этом можно прочитать, например, [в этой статье](https://treyhunner.com/2018/02/python-range-is-not-an-iterator/)

In [44]:
range(5)

range(0, 5)

In [45]:
iter(range(5))

In [46]:
for i in range(5):
    print(i)

0
1
2
3
4
